### Pre-requisites:

In [1]:
#Libraries for time computation and data importation:
import time
import os

#Libraries to manage the Interaction Matrix:
import pandas as pd
import numpy as np

#Utility Functions:
import utils
from utils import *

In [4]:
#Data Structures:

import DataStructures
from DataStructures import MaxHeap

### Algorithms:

In [5]:
#Select only users with a certain number of features to make the comparison:
def select(data, target, threshold):
    """
    FUNCTION:
    Function to select only the users with a certain percentage of comparisons they can make with the target user.
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    target -> Target user.
    threshold -> Percentage of comparable items required.
    
    RETURNS:
    Pandas RangeIndex containing the selected user's indexes.
    List with the columns names of the items rated by the target user.
    """
    #Extract the items that the target user has rated
    valid = data.iloc[target][np.isnan(data.iloc[target]) == False].index
    r = len(valid)
    
    #For each row (user) except for the target one, compute how many of the "valid" items has been seen
    unmatches = data[valid].drop(target).isnull().sum(axis = 1)
    
    #Define which are the "acceptable" users
    acceptable = unmatches[unmatches <= round((1 - threshold)*r, 0)].index
    
    if len(acceptable) == 0:
        return data.drop(target).index
    
    return (acceptable, valid)
    
#Algorithm to order the nearest neighbors:
def nearest(data, target, threshold):
    """
    FUNCTION:
    Order the nearest users to the target user. The function will consider only neighbors extracted by the 'select()' function.
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    target -> Target user.
    
    RETURNS:
    Priority Queue containing the indeces of the nearest neighbors to the target user.
    List with the columns names of the items rated by the target user.
    """
    #Extract only the acceptable users to perform the recommendation
    indexes, valid = select(data, target, threshold)
    
    #Computes distances and insert them in a Priority Queue
    distances = MaxHeap()
    for user in indexes:
        distances.insert(cosine(data.iloc[target].values, data.iloc[user].values), user)
            
    return (distances, valid)

#Assign a predicted rating to an unseen item:
def compute_ratings(data, neighbors, valid, target, k = 10):
    """
    FUNCTION:
    Compute the ratings for the target user's unseen items considering the average ratings of its k-neighbors for those items.
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    neighbors -> Priority Queue with the k-neighbors of the target user.
    valid -> List with the columns names of the items rated by the target user.
    target -> Target user.
    k -> Number of neighbors.
    
    RETURNS:
    Priority Queue of rating's values for the target user. 
    """
    #Priority Queue for the item's predicted ratings
    predictions = list()
    
    #Items not rated by the target user
    unseen = data.columns.drop(valid)
    
    #K-nearest neighbors:
    nearest = list()
    for iteration in range(k):
        nearest.append(neighbors.delete()[1])
    
    #For each item already not rated
    for item in unseen:
        
        #Variables to store the predicted rating
        rating = False
        n_comparison = 0
        
        #We select the k-neighbors
        for iteration in range(k):
            neighbor = nearest[iteration]
            
            #Check if the unseen item has been rated by the neighbors
            if item in data.iloc[neighbor][np.isnan(data.iloc[neighbor]) == False].index:
                
                #Compute the predicted rating
                rating += data.iloc[neighbor][item]
                n_comparison += 1
                
        #Eventually, add it to the Priority Queue of predicted ratings
        if rating:
            rating = rating/n_comparison
            predictions.append((rating, item))
    
    return predictions

#Return suggestions:
def suggest(target, ratings):
    """
    FUNCTION:
    Assign a recommendation label to each item.
    
    PARAMETERS:
    target -> Target user.
    ratings -> Priority Queue containing the expected ratings for each unseen evaluable item.
    
    RETURNS:
    Dictionary containing the recomendations for the target user.
    """
    #Dictionary to store items according to their class of recommendation
    recommandations = {
        "Strongly Recommended" : list(),
        "Recommended" : list(),
        "Not Recommended" : list()
    }
    
    #Loop through all the unseen items for which a rating has been predicted
    while len(ratings) > 0:
        rating, item = ratings.pop()
        
        #Rule defined for the recommendation class assignment
        if rating >= 4:
            recommandations["Strongly Recommended"].append(item)
        elif rating >= 3:
            recommandations["Recommended"].append(item)
        else:
            recommandations["Not Recommended"].append(item)
    
    return recommandations

def recommend(data, target, k = 10, threshold = 0.33):
    """
    FUNCTION:
    Perform in a single command all the previous algorithms necessary to obtain the recommendation.
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    target -> Target user.
    k -> Number of neighbors.
    threshold -> Percentage of comparable items required.
    
    RETURNS:
    Dictionary containing the recomendations for the target user.
    """
    neighbors, valid = nearest(data, target, threshold = threshold)
    ratings = compute_ratings(data, neighbors, valid, target, k = k)
    return suggest(target, ratings)

### Test on real data:

In [6]:
#Data importation:
path = os.path.join(os.getcwd(), "data_s.csv")
df = pd.read_csv(path)
df

,Toy Story,Twelve Monkeys,Se7en,The Usual Suspects,Braveheart,Apollo 13,Star Wars,Pulp Fiction,The Shawshank Redemption,Forrest Gump,...,Independence Day,The Godfather,The Empire Strikes Back,Raiders of the Lost Ark,Return of the Jedi,Back to the Future,The Matrix,American Beauty,Fight Club,The Lord of the Rings: The Fellowship of the Ring
0,NaN,NaN,4.0,4.0,4.0,5.0,NaN,4.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.5,5.0,5.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,5.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,5.0,...,NaN,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,...,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,5.0,NaN,5.0,NaN,5.0,3.0,...,3.0,NaN,5.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,NaN,4.0,NaN,NaN,NaN,NaN,4.0,4.0,5.0,4.0,...,NaN,NaN,4.0,NaN,4.0,5.0,5.0,3.0,5.0,5.0
485,NaN,NaN,NaN,NaN,2.0,4.0,NaN,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,NaN,5.0,NaN,NaN,4.0,4.0,NaN,5.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,4.0,3.0,5.0,5.0,3.0,3.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN


In [8]:
#Test the algorithms on one user:
print("Suggestions for user: " + str(3))
neighbors, valid = nearest(df, 3, threshold = 0.33)
ratings = compute_ratings(df, neighbors, valid, 3)
suggest(3, ratings)

Suggestions for user: 3


{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring',
  'Fight Club',
  'The Matrix',
  'Back to the Future',
  'Return of the Jedi',
  'Raiders of the Lost Ark',
  'The Empire Strikes Back',
  'Fargo',
  'The Silence of the Lambs',
  'Dances with Wolves',
  'Terminator 2: Judgment Day',
  "Schindler's List",
  'The Shawshank Redemption',
  'Pulp Fiction',
  'Star Wars',
  'The Usual Suspects',
  'Twelve Monkeys',
  'Toy Story'],
 'Recommended': ['American Beauty',
  'Batman',
  'Jurassic Park',
  'The Fugitive',
  'True Lies',
  'Braveheart',
  'Se7en'],
 'Not Recommended': ['Independence Day']}

In [10]:
#Test the algorithms for all the users:
start_time = int(time.ctime().split()[3].split(":")[0])*60*60 + int(time.ctime().split()[3].split(":")[1])*60 + int(time.ctime().split()[3].split(":")[2])

for target in df.index:
    print("Suggestions for user: " + str(target))
    neighbors, valid = nearest(df, target, threshold = 0.33)
    ratings = compute_ratings(df, neighbors, valid, target)
    print(suggest(target, ratings))
    print("-"*125)

current = int(time.ctime().split()[3].split(":")[0])*60*60 + int(time.ctime().split()[3].split(":")[1])*60 + int(time.ctime().split()[3].split(":")[2])
print("\nTotal time required: " + str(current - start_time) + "s")

Suggestions for user: 0
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'The Shawshank Redemption', 'Star Wars', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['American Beauty', 'Return of the Jedi', 'Independence Day', 'True Lies'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 1
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'The Matrix', 'Back to the Future', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'Terminator 2: Judgment Day', 'The Lion King', 'Star Wars', 'Apollo 13', 'The Usual Suspects', 'Se7en', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['Independence Day', 'Dances with Wolves', 'Jurassic Park', 'The Fugitive', 'True Li

{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'Terminator 2: Judgment Day', 'The Lion King', 'Forrest Gump', 'The Shawshank Redemption', 'Star Wars', 'Apollo 13', 'Se7en', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['Batman', 'Dances with Wolves', 'Aladdin', 'Jurassic Park', 'The Fugitive', 'True Lies', 'Braveheart'], 'Not Recommended': ['Independence Day']}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 19
{'Strongly Recommended': [], 'Recommended': [], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 20
{'Strongly Recommended': ['Back to the Future', 'Return of the Jedi', 'Raider

{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'The Silence of the Lambs', 'The Lion King', 'The Shawshank Redemption', 'Star Wars', 'The Usual Suspects', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['American Beauty', 'Batman', 'Aladdin'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 34
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'The Silence of the Lambs', 'Terminator 2: Judgment Day', "Schindler's List", 'The Lion King', 'The Shawshank Redemption', 'Pulp Fiction', 'Apollo 13', 'The Usual Suspects', 'Se7en'], 'Recommended':

{'Strongly Recommended': ['Return of the Jedi', 'Fargo', 'True Lies', 'The Lion King', 'Pulp Fiction', 'Apollo 13', 'The Usual Suspects', 'Se7en'], 'Recommended': ['American Beauty', 'Independence Day', 'The Silence of the Lambs', 'Dances with Wolves', 'Aladdin', 'The Fugitive', 'Braveheart'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 50
{'Strongly Recommended': ['The Godfather', "Schindler's List", 'Apollo 13', 'The Usual Suspects'], 'Recommended': [], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 51
{'Strongly Recommended': ['Raiders of the Lost Ark', 'Fargo', "Schindler's List"], 'Recommended': ['Batman', 'Dances with Wolves', 'The Fugitive', 'True Lies', 'Toy Story'], 'Not Recommended': []}
------------------------------------------

{'Strongly Recommended': ['The Matrix', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'Dances with Wolves', 'Terminator 2: Judgment Day', "Schindler's List", 'Forrest Gump', 'The Shawshank Redemption', 'Apollo 13', 'The Usual Suspects', 'Twelve Monkeys'], 'Recommended': ['Back to the Future', 'Batman', 'Aladdin', 'True Lies', 'Braveheart', 'Se7en', 'Toy Story'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 67
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'The Silence of the Lambs', 'Terminator 2: Judgment Day', 'The Lion King', 'Forrest Gump', 'Pulp Fiction', 'Star Wars', 'Apollo 13', 'Se7en', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['Independence Day', 'Batman', 'Dances with

...

{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'The Matrix', 'Return of the Jedi', 'The Empire Strikes Back', 'Fargo', "Schindler's List", 'The Lion King', 'Pulp Fiction', 'Apollo 13', 'Se7en', 'Twelve Monkeys'], 'Recommended': ['Back to the Future', 'Batman', 'Dances with Wolves', 'Aladdin', 'Jurassic Park', 'The Fugitive', 'True Lies', 'Braveheart', 'Toy Story'], 'Not Recommended': ['Independence Day']}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 414
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', "Schindler's List", 'The Lion King', 'Star Wars', 'The Usual Suspects', 'Twelve Monkeys'], 'Recommended': ['American Beauty', 'Independence Day', 'Braveheart'], 'Not Recommende

{'Strongly Recommended': ['Back to the Future', 'Return of the Jedi', 'The Empire Strikes Back', 'Terminator 2: Judgment Day', 'The Lion King', 'Forrest Gump', 'Star Wars', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['Batman', 'Dances with Wolves', 'Aladdin', 'Jurassic Park', 'True Lies', 'Braveheart'], 'Not Recommended': ['Independence Day']}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 430
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'Star Wars', 'Toy Story'], 'Recommended': ['American Beauty', 'Independence Day', 'Aladdin'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 43

{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Return of the Jedi', 'The Empire Strikes Back', 'The Lion King', 'The Shawshank Redemption', 'Star Wars', 'Apollo 13'], 'Recommended': ['Back to the Future', 'Dances with Wolves', 'Aladdin', 'Jurassic Park', 'True Lies', 'Braveheart'], 'Not Recommended': ['Independence Day']}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 447
{'Strongly Recommended': ['Back to the Future', 'The Empire Strikes Back', 'Fargo', 'Terminator 2: Judgment Day', "Schindler's List", 'True Lies', 'The Lion King', 'Forrest Gump', 'Pulp Fiction', 'Apollo 13', 'Se7en', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['Return of the Jedi', 'Independence Day', 'Batman', 'Dances with Wolves', 'Aladdin', 'Jurassic Park', 'The Fugitive', 'Braveheart'], 'Not Recommended': []}
---------------------------------------------------------------------

{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Back to the Future', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'The Shawshank Redemption', 'Star Wars', 'The Usual Suspects', 'Se7en', 'Twelve Monkeys', 'Toy Story'], 'Recommended': ['American Beauty', 'Independence Day', 'Batman'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 461
{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'Fargo', 'Terminator 2: Judgment Day', "Schindler's List", 'True Lies', 'The Lion King', 'The Shawshank Redemption', 'Star Wars', 'Apollo 13', 'The Usual Suspects', 'Se7en', 'Twelve Monkeys'], 'Recommended': ['American Beauty', 'Independence Day', 'Batman

{'Strongly Recommended': ['The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'The Matrix', 'Return of the Jedi', 'Raiders of the Lost Ark', 'The Empire Strikes Back', 'The Godfather', 'Fargo', 'Terminator 2: Judgment Day', "Schindler's List", 'The Shawshank Redemption', 'Pulp Fiction', 'Star Wars', 'The Usual Suspects', 'Se7en', 'Twelve Monkeys'], 'Recommended': ['American Beauty', 'Back to the Future', 'Independence Day', 'The Silence of the Lambs', 'Batman', 'Jurassic Park', 'True Lies', 'Braveheart'], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 475
{'Strongly Recommended': [], 'Recommended': [], 'Not Recommended': []}
-----------------------------------------------------------------------------------------------------------------------------
Suggestions for user: 476
{'Strongly Recommended': ['American Beauty', 'Return of the Jedi', 'The Godf